# Pruebas sobre el dataset dev

## Primera parte. Entrenar modelo sobre dataset completo

In [1]:
import pandas as pd
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..', 'src')))


In [2]:
# Cargar el conjunto de entrenamiento
df = pd.read_csv('../data/TA1C_dataset_detection_train.csv')
df = df.drop(['Tweet Date', 'Media Name', 'Media Origin'], axis=1)

In [3]:
from preprocessing import normalize_text

df["cleaned_lemmatized_text"] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="text_cleaning")).apply(lambda x: normalize_text(x, mode="lemmatization"))
# Si cada celda es una lista de tokens, conviértela a string
df['cleaned_lemmatized_text'] = df['cleaned_lemmatized_text'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else tokens)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Probar diferentes configuraciones de n-gramas y representaciones
configurations = [
    {"ngram_range": (1, 1), "representation": "tfidf"},  # Unigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "tfidf"},  # Unigramas + Bigramas con TF-IDF
    {"ngram_range": (1, 3), "representation": "tfidf"},  # Unigramas + Bigramas + Trigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "binary"},  # Unigramas + Bigramas con representación binaria
    {"ngram_range": (1, 2), "representation": "frequency"},  # Unigramas + Bigramas con frecuencia
]

for config in configurations:
    print(f"Configuración: ngram_range={config['ngram_range']}, representación={config['representation']}")
    
    # Configurar el vectorizador
    if config["representation"] == "tfidf":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"])
    elif config["representation"] == "binary":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], binary=True)
    elif config["representation"] == "frequency":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], use_idf=False)
    
    # Crear representaciones para el conjunto de entrenamiento y desarrollo
    X = vectorizer.fit_transform(df['cleaned_lemmatized_text'])
    
    # Verificar las dimensiones
    print(f"Dimensiones de X: {X.shape}")

Configuración: ngram_range=(1, 1), representación=tfidf
Dimensiones de X: (2800, 10584)
Configuración: ngram_range=(1, 2), representación=tfidf
Dimensiones de X: (2800, 47785)
Configuración: ngram_range=(1, 3), representación=tfidf
Dimensiones de X: (2800, 98926)
Configuración: ngram_range=(1, 2), representación=binary
Dimensiones de X: (2800, 47785)
Configuración: ngram_range=(1, 2), representación=frequency
Dimensiones de X: (2800, 47785)


In [5]:
from sklearn.decomposition import TruncatedSVD

# Reducir dimensionalidad con TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=0)  # Reducir a 100 dimensiones
X_reduced = svd.fit_transform(X)

# Verificar las dimensiones después de la reducción
print(f"Dimensiones reducidas de X_reduced: {X_reduced.shape}")

Dimensiones reducidas de X_reduced: (2800, 100)


In [6]:
# Convertir etiquetas de texto a valores numéricos
label_mapping = {'Clickbait': 1, 'No': 0}
df['Tag Value'] = df['Tag Value'].map(label_mapping)
y = df['Tag Value']

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

# Definir el modelo de MLP
mlp_model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=2000, alpha=0.001, solver='adam', random_state=0)

# Evaluar el modelo con validación cruzada usando las representaciones reducidas (X_train_reduced)
mlp_scores = cross_val_score(mlp_model, X_reduced, y, cv=5, scoring='f1_macro')

# Imprimir los resultados
print("Multi-layer Perceptron (MLPClassifier):")
print(f"F1-macro scores por fold: {mlp_scores}")
print(f"F1-macro promedio: {mlp_scores.mean():.4f}")

Multi-layer Perceptron (MLPClassifier):
F1-macro scores por fold: [0.67875994 0.72960786 0.67733244 0.77074631 0.70552885]
F1-macro promedio: 0.7124
